| Übungszettel 8 | Einführung in die Numerik |
|---|---|
| Dozent      | Prof. Kanschat |
| Tutoren     | S. Meggendorfer und J. Witte |
| Abgabe      | bis 17.06.21 23.15 Uhr |
| Studierende | Daniel Schiller, Thomas Blankenburg, Dennis Jakob |


---

### Aufgabe 1: Summierte Quadratur

Schreiben Sie eine Funktion, die die Funktion $f(x) = \sin(\pi x)$ über das Intervall $[0,1]$ mit der iterierten Trapezregel integriert. Geben Sie den approximierten Integralwert, sowie den Fehler für verschiedene Schrittweiten $h= 1,\tfrac12,\tfrac14,\tfrac18,...$ aus.

### Aufgabe 2: Romberg-Quadratur

Wenden Sie die Romberg-Quadratur mit der Schrittfolge $h= 1,\tfrac12,\tfrac14,\tfrac18,...$
auf die iterierte Trapezregel aus Aufgabe 1 an und beobachten Sie die Konvergenz gegen den exakten Integralwert $\frac{2}{\pi}$ für die verschiedenen Spalten im Tableau des Neville-Algorithmus. Lassen Sie sich auch hierfür wieder den Fehler ausgeben und vergleichen Sie den Aufwand der Romberg-Quadratur mit der iterierten Trapezregel.

### Bonus: Konvergenzraten

Die Konvergenzrate $p$ eines numerischen Verfahrens, dessen Fehler $e(h)$ eine Abschätzung $e(h)= ch^p + o(h^p)$ erlaubt, kann experimentell bestimmt werden mit der Formel
$$p \approx \log_2(e(h)) - log_2(e({\tfrac{h}2})).$$
Berechnen Sie mit dieser Formel die Konvergenzraten im Tableau des Neville-Algorithmus aus Aufgabe 2.

**Hinweis:** Der Logarithmus zur Basis 2 lässt sich in numpy mit dem Befehl `np.log2()` berechnen.

In [1]:
import numpy as np

---

Aufgabe 1:

In [2]:
def Q_Trapez(n):
    res = 0.
    h = 1. / float(n)
    w_n = h / 2.
    for i in range(int(n)):
        res += w_n * np.sin(np.pi * float(i) * h) + w_n * np.sin(np.pi * (float(i) + 1.) * h)
    return res

In [3]:
Q_richtig = 2. / np.pi

In [4]:
for j in range(5):
    n = 2.**j
    print("n = ", n)
    h = 1. / n
    print("h = ", h)
    Q_Trap_n = Q_Trapez(n)
    print("Q_Trap_n = ", Q_Trap_n)
    Fehler_n = np.abs(Q_Trap_n - Q_richtig)
    print("Fehler_n = ", Fehler_n)
    print()

n =  1.0
h =  1.0
Q_Trap_n =  6.123233995736766e-17
Fehler_n =  0.6366197723675813

n =  2.0
h =  0.5
Q_Trap_n =  0.5
Fehler_n =  0.13661977236758138

n =  4.0
h =  0.25
Q_Trap_n =  0.6035533905932737
Fehler_n =  0.03306638177430765

n =  8.0
h =  0.125
Q_Trap_n =  0.628417436515731
Fehler_n =  0.008202335851850373

n =  16.0
h =  0.0625
Q_Trap_n =  0.6345731492255539
Fehler_n =  0.002046623142027526



Aufgabe 2:

In [5]:
def h(i):
    return 2.**(-i)

def Romberg(i, k):
    if k == 0:
        return Q_Trapez(int(2.**(float(i))))
    else:
        return Romberg(i, k - 1) + (h(i)**2 / (h(i - k)**2 - h(i)**2)) * (Romberg(i, k - 1) - Romberg(i-1, k-1))    

In [6]:
from tabulate import tabulate

tab_Romberg = []
tab_Fehler = []
tab_konvergenzrate = [""]
for i in range(5):
    row = []
    row_Fehler = []
    for j in range(5):
        if i >= j:
            row.append(Romberg(i, j))
            row_Fehler.append(2. / np.pi - Romberg(i, j))
        else:
            row.append("")
            row_Fehler.append("")
    if i: tab_konvergenzrate.append([np.log2(np.abs(float(tab_Fehler[-1][i - 1]))) - np.log2(np.abs(float(row_Fehler[i])))])
    tab_Romberg.append(row)
    tab_Fehler.append(row_Fehler)

print(tabulate(tab_Romberg, headers = ["Q_i0", "Q_i1", "Q_i2", "Q_i3", "Q_i4"]))
print()
print(tabulate(tab_Fehler, headers = ["Fehler_i0", "Fehler_i1", "Fehler_i2", "Fehler_i3", "Fehler_i4"]))
print()
print(tabulate(tab_konvergenzrate, headers = ["konvergenzrate"]))

       Q_i0  Q_i1                Q_i2                Q_i3                Q_i4
-----------  ------------------  ------------------  ------------------  ------------------
6.12323e-17
0.5          0.6666666666666666
0.603553     0.6380711874576983  0.6361648221771005
0.628417     0.6367054518232168  0.6366144027809181  0.6366215389809786
0.634573     0.6366250534621615  0.6366196935714244  0.6366197775522261  0.6366197706446624

  Fehler_i0  Fehler_i1               Fehler_i2               Fehler_i3                Fehler_i4
-----------  ----------------------  ----------------------  -----------------------  ----------------------
 0.63662
 0.13662     -0.030046894299085247
 0.0330664   -0.0014514150901169254  0.00045495019048091834
 0.00820234  -8.567945563542345e-05  5.369586663306336e-06   -1.7666133972360498e-06
 0.00204662  -5.281094580089629e-06  7.879615693262565e-08   -5.184644713551734e-09   1.7229190296674801e-09

  konvergenzrate
----------------

         4.40514
         6.04

Für die Spalten weiter rechts konvergiert der Fehler schneller gegen 0.
Die Konvergenzrate des rechten wertes beschleunigt mit jeder zeile.

Bei der Romberg-Quadratur müssen die vorherigen Quadraturen mit größerer Schrittweitere der iterieteren Trapezquadratur ausgeführt werden. Darum sollte der Aufwand der Romberg-Quadratur quadratisch steigen während der Aufwand der iterierten Trapezquadratur linear ist.